### Preppin' Data Challenge 2024: Week 5 - Getting the right data
Input the data

In [193]:
import pandas as pd
import numpy as np

df1 = pd.read_csv('PD2024W5_PREPAIR_CUSTOMERS.csv')
df2 = pd.read_csv('PD2024W5_PREPAIR_FLIGHTS.csv')
df3 = pd.read_csv('PD2024W5_PREPAIR_TICKETSALES.csv')

<br>
For the first output:
    
- Create a dataset that gives all the customer details for booked flights in 2024. Make sure the output also includes details on the flights origin and destination
- When outputting the data, create an excel file with a new sheet for each output (so 1 file for all outputs this week!)

In [194]:
#Performing inner join
Fdf = pd.merge (df1, df3, on = 'Customer ID', how='inner')

In [195]:
#Performing inner join (joining three tables)
Fdf = pd.merge(Fdf, df2, on = ['Date', 'Flight Number'], how = 'inner')

In [196]:
#Rearranging the column as per the desired output
FirstOutput = Fdf[[
        'Date', 'From', 'To', 'Flight Number','Customer ID',
        'Last Date Flown','first_name','last_name','email','gender','Ticket Price'
         ]]

For the second output:

- Create a dataset that allows Prep Air to identify which flights have not yet been booked in 2024
- Add a datestamp field to this dataset for today's date (31/01/2024) so that Prep Air know the unbooked flights as of the day the workflow is run
- When outputting the table to a new sheet in the Excel Workbook, choose the option "Append to Table" under Write Options. This means that if the workflow is run on a different day, the results will add additional rows to the dataset, rather than overwriting the previous run's data

In [197]:
#Performing Left join
Sdf = pd.merge(df2, df3, on = ['Date', 'Flight Number'], how='left')

#Retaining records from only left table
Sdf = Sdf[Sdf['Customer ID'].isna()]

Sdf['Flights unbooked as of'] = pd.to_datetime('2024-1-31').date()

In [198]:
#Rearranging the column as per the desired output
SecondOutput = Sdf[['Flights unbooked as of','Date','Flight Number','From','To']]

For the third output:

- Create a dataset that shows which customers have yet to book a flight with Prep Air in 2024
- Create a field which will allow Prep Air to see how many days it has been since the customer last flew (compared to 31/01/2024)
- Categorise customers into the following groups:
    - Recent fliers - flown within the last 3 months
    - Taking a break - 3-6 months since last flight
    - Been away a while - 6-9 months since last flight
    - Lapsed Customers - over 9 months since last flight


In [199]:
#Performing Left join
Tdf = pd.merge (df1, df3, on = 'Customer ID', how='left')

#Retaining records from only left table
Tdf = Tdf[Tdf['Flight Number'].isna()]



In [200]:
#Assigning hardcoded date value in Today column
Tdf['Today'] = pd.to_datetime('2024-01-31')
Tdf['Last Date Flown'] = pd.to_datetime(Tdf['Last Date Flown'])

#Performing the date difference to output days as integer
Tdf['Days Since Last Flown'] = (Tdf['Today'] - Tdf['Last Date Flown']).dt.days


In [201]:
#Categorizing as per the given condition
Tdf['Customer Category'] = Tdf['Days Since Last Flown'].apply(lambda x: 'Recent fliers' if x < 90
                               else 'Taking a break' if x >=90 & x <180
                               else 'Been away a while' if x >=180 & x <270
                               else 'Lapsed Customers')

In [202]:
#Rearranging the column as per the desired output
ThirdOutput = Tdf[['Customer ID','Customer Category','Days Since Last Flown',
                   'Last Date Flown','first_name','last_name','email','gender',]]

Output the data to a new sheet in the Excel Workbook

In [203]:
#Using ExcelWriter to export three different DataFrames to separate sheets 
with pd.ExcelWriter('PD2024Week5_Output.xlsx') as writer:
    FirstOutput.to_excel(writer, sheet_name='First', index=False)
    SecondOutput.to_excel(writer, sheet_name='Second', index=False)
    ThirdOutput.to_excel(writer, sheet_name='third', index=False)

In [204]:
#verifying the output
Output = pd.read_excel('PD2024Week5_Output.xlsx', sheet_name=None)
Output1 = Output['First']
Output2 = Output['Second']
Output3 = Output['third']

In [205]:
Output1

,Date,From,To,Flight Number,Customer ID,Last Date Flown,first_name,last_name,email,gender,Ticket Price
0,2024-06-03,London,Perth,PA003,1,2023-01-05,Denyse,Gebuhr,dgebuhr0@vinaora.com,Female,1430.99
1,2024-06-03,London,Perth,PA003,676,2023-11-06,Florri,McCuaig,fmccuaigir@networksolutions.com,Female,1839.99
2,2024-06-03,London,Perth,PA003,1038,2023-04-22,Fabien,Noller,fnoller11@discuz.net,Male,331.99
3,2024-06-03,London,Perth,PA003,1245,2023-01-30,Alic,O'Dulchonta,aodulchonta6s@mashable.com,Male,1786.99
4,2024-06-03,London,Perth,PA003,1398,2023-09-01,Edythe,Boutcher,eboutcherb1@skype.com,Female,327.99
...,...,...,...,...,...,...,...,...,...,...,...
44763,2024-11-19,New York,Perth,PA007,9076,2023-07-19,Silvio,Rosenshine,srosenshine23@dion.ne.jp,Male,1996.99
44764,2024-11-18,Perth,Tokyo,PA011,9109,2023-11-10,Marian,Yukhtin,myukhtin30@amazon.com,Female,1930.99
44765,2024-11-18,Tokyo,Perth,PA012,9109,2023-11-10,Marian,Yukhtin,myukhtin30@amazon.com,Female,187.99
44766,2024-11-14,London,New York,PA001,9481,2023-01-28,Minne,Dracksford,mdracksforddc@comcast.net,Female,1728.99


In [206]:
Output2

,Flights unbooked as of,Date,Flight Number,From,To
0,2024-01-31,2024-11-22,PA001,London,New York
1,2024-01-31,2024-11-23,PA001,London,New York
2,2024-01-31,2024-11-23,PA002,New York,London
3,2024-01-31,2024-11-24,PA001,London,New York
4,2024-01-31,2024-11-27,PA001,London,New York
...,...,...,...,...,...
291,2024-01-31,2024-12-27,PA012,Tokyo,Perth
292,2024-01-31,2024-12-28,PA012,Tokyo,Perth
293,2024-01-31,2024-12-29,PA011,Perth,Tokyo
294,2024-01-31,2024-12-30,PA011,Perth,Tokyo


In [207]:
Output3

,Customer ID,Customer Category,Days Since Last Flown,Last Date Flown,first_name,last_name,email,gender
0,3,Recent fliers,83,2023-11-09,Tyler,McGrail,tmcgrail2@nyu.edu,Male
1,9,Taking a break,240,2023-06-05,Binnie,Jeckell,bjeckell8@123-reg.co.uk,Female
2,14,Taking a break,274,2023-05-02,Ashil,Tetlow,atetlowd@woothemes.com,Female
3,15,Taking a break,258,2023-05-18,Ayn,Bengtson,abengtsone@bloomberg.com,Female
4,24,Taking a break,350,2023-02-15,Grace,Piesing,gpiesingn@zdnet.com,Female
...,...,...,...,...,...,...,...,...
1255,9972,Taking a break,109,2023-10-14,Any,Mongeot,amongeotqz@twitter.com,Male
1256,9975,Taking a break,333,2023-03-04,Idell,Clyant,iclyantr2@hostgator.com,Female
1257,9978,Taking a break,285,2023-04-21,Spike,Thaller,sthallerr5@gov.uk,Agender
1258,9989,Recent fliers,86,2023-11-06,Robbie,Grugerr,rgrugerrrg@plala.or.jp,Female
